In [19]:
import os, shutil
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split

In [206]:
# original data directories
myPaths = ['../Data/CT/', '../Data/MRI/', '../Data/PET/']
myDict={'../Data/CT/':[], '../Data/MRI/':[], '../Data/PET/':[]}

for path in myPaths:
    myDict[path] = [f for f in listdir(path) if isfile(join(path, f))]
    #print(len(myDict[path]))
    #print(myDict[path])

#split train test
CT_train, CT_test = train_test_split(myDict[myPaths[0]], train_size=0.8)
MRI_train, MRI_test = train_test_split(myDict[myPaths[1]], train_size=0.8)
PET_train, PET_test = train_test_split(myDict[myPaths[2]], train_size=0.8)

#split train validation
#CT_train, CT_validation = train_test_split(CT_train, train_size=0.75)
#MRI_train, MRI_validation = train_test_split(MRI_train, train_size=0.75)
#PET_train, PET_validation = train_test_split(PET_train, train_size=0.75)

#print(len(MRI_train))
#print(len(MRI_validation))
#print(len(MRI_test))


# unisco
#train = CT_train + MRI_train + PET_train
#validation = CT_validation + MRI_validation + PET_validation
#test = CT_test + MRI_test + PET_test

#print(CT_train)
#print(len(validation))
#print(len(test))

In [242]:
x = io.imread('data/test/PET200.png') 
for img in listdir('data/test'):
    print('data/test/'+img)
    y = io.imread('data/test/'+img)
    x = np.array([x,y])
    print(x.shape)



data/test/PET200.png
(2, 182, 218)
data/test/MRI4.png
(2,)
data/test/CT343.png
(2,)
data/test/CT196.png
(2,)
data/test/MRI405.png
(2,)
data/test/CT27.png
(2,)
data/test/MRI39.png
(2,)
data/test/CT33.png
(2,)
data/test/PET55.png
(2,)
data/test/PET96.png
(2,)
data/test/CT237.png
(2,)
data/test/PET83.png
(2,)
data/test/PET54.png
(2,)
data/test/MRI38.png
(2,)
data/test/MRI389.png
(2,)
data/test/CT140.png
(2,)
data/test/MRI5.png
(2,)
data/test/PET229.png
(2,)
data/test/MRI7.png
(2,)
data/test/MRI200.png
(2,)
data/test/MRI214.png
(2,)
data/test/MRI406.png
(2,)
data/test/CT30.png
(2,)
data/test/CT24.png
(2,)
data/test/PET42.png
(2,)
data/test/CT18.png
(2,)
data/test/CT208.png
(2,)
data/test/MRI176.png
(2,)
data/test/MRI162.png
(2,)
data/test/PET94.png
(2,)
data/test/MRI163.png
(2,)
data/test/CT209.png
(2,)
data/test/CT157.png
(2,)
data/test/MRI201.png
(2,)
data/test/CT355.png
(2,)
data/test/PET206.png
(2,)
data/test/PET212.png
(2,)
data/test/CT345.png
(2,)
data/test/CT351.png
(2,)
data/test/C

In [122]:
# creo cartella data
#print(os.getcwd())

directories = ['data/train', 
               #'data/validation', 
               'data/test']

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

myPaths = ['../Data/CT/', '../Data/MRI/', '../Data/PET/']
scans = ['CT', 'MRI', 'PET']

for dest in dests:
    for source, scan in zip(myPaths, scans):
        files = eval(scan+'_test')
        #print(files)
        for f in files:
            shutil.copy(source+f, dest)

In [188]:
import numpy as np

def load_mnist():
    # the data, shuffled and split between train and test sets
    from keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    print(x_train.shape)
    x = np.concatenate((x_train, x_test))
    
    y = np.concatenate((y_train, y_test))
    x = x.reshape(-1, 28, 28, 1).astype('float32')
    x = x/255.
    print('MNIST:', x.shape)
    return x, y

load_mnist()


(60000, 28, 28)
MNIST: (70000, 28, 28, 1)


(array([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
      

In [ ]:
CT = np.concatenate((CT_train,CT_test))
MRI = np.concatenate((MRI_train,MRI_test))

LabelDict = {'CT':CT, 'MRI':MRI}
CT.shape